# Assignment #9 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2025</br>

Name: Niyati Parekh
</br>
Date: 4/20/25
<br>
<br>
**At this time in the semester:** <br>
- We have explored a dataset. <br>
- We have cleaned our dataset. <br>
- We created a Github account with a repository for this class and included a metadata read me file about our data. <br>
- We introduced general SQL syntax, queries, and applications in Python.<br>
- Created our own databases from scratch using MySQL Workbench and Python with SQLAlchemy on our local server and locally on our machine.
- Populated our databases with the data we cleaned at the start of the semester.
<br>

At this point, we have discussed all major statements used with SQL, but the possibilities are endless when it comes to data! Below we will explore some miscellaneous statements and tools that may be useful with your database.<br>

<br>

Read Chapter 7 & 10 in Getting Started with SQL by Thomas Nield available on Blackboard. <br>
A quick reference for SQL commands: https://www.w3schools.com/sql/default.asp <br>

Review the powerpoint and other readings specified on Blackboard in the Discussion Board.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source, especially when posting your assignments to Github.<br>

Feel free to use your preferred library and method for the exploration below. Now that all of our data is loaded onto the MySQL Workbench server, you can even take the assignment a step further and complete it all within SQL without Jupyter Notebook!<br>

Follow the instructions below to complete the assignment. Be sure to comment **all** code and answer **all** questions in markdown for full credit. Please submit this assignment with a link to it posted to your Github.<br>

**Data origin:** 

In [3]:
# Load necessary packages:
from sqlalchemy import create_engine, inspect, Column, String, Integer, Boolean, BigInteger, Float, text # Database navigation
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import mysql.connector
import sqlite3 
import pandas as pd # Python data manilpulation
import numpy as np # Populating tables 

In [108]:
# Setting working directory 
import os
os.chdir(r"C:\Users\parni\OneDrive\Desktop\DSSA\DGW")

In [7]:
#connect to database file using SQLite
db_path = r"C:\Users\parni\OneDrive\Desktop\DSSA\DGW\sql-murder-mystery (3).db" 
engine = create_engine(f"sqlite:///{db_path}") # Use the create_engine function to connect to the database
print(engine) 

Engine(sqlite:///C:\Users\parni\OneDrive\Desktop\DSSA\DGW\sql-murder-mystery (3).db)


In [9]:
#make a connection to the database using the engine
with engine.connect() as connection:
    #if the connection is successful print the statement  
    print("Connection successful!")

Connection successful!


In [13]:
# Write a query to list the table names of the database:

#create an inspector object to analyze the databse
inspector = inspect(engine)

#print table names using a built in function 
table_names = inspector.get_table_names()
print(table_names)

['crime_scene_report', 'drivers_license', 'facebook_event_checkin', 'get_fit_now_check_in', 'get_fit_now_member', 'income', 'interview', 'person', 'solution']


In [15]:
#store a cleaner variable of all table names 
#replaces the underscore with a space and capitalizes each word
clean_table_names = [name.replace('_', ' ').title() for name in table_names]

#print a statement of all table names 
print("The table names are:", clean_table_names)

The table names are: ['Crime Scene Report', 'Drivers License', 'Facebook Event Checkin', 'Get Fit Now Check In', 'Get Fit Now Member', 'Income', 'Interview', 'Person', 'Solution']


#### CASE Statements
Case statements are similar to the if-then-else statements in programming. The data that meets the case statements in the database will be returned. You start a CASE statement with the word CASE and conclude it with an END. Between those keywords, you specify each condition with a WHEN [condition] THEN [value], where the condition and the values are specified by you.

Nield, Thomas. Getting Started with SQL (p. 71). O'Reilly Media. Kindle Edition. <br>
<br>
Write your question you are answering with your data query. <br>
<br>
**Example Question:** Are people in SQL city lower income, middle-income, or upper income households?
<br>
**What tables are we joining? (If any):** people and income tables 

In [19]:
#make a connection to the database using the engine
with engine.connect() as connection:
    #define the query to join the people and income tables and then classify each person's income as lower (< $15k),middle (>= $16k and ,$75k), or upper level income (>$75k) 
    query = text("""
        SELECT
            person.name,
            income.ssn,
            income.annual_income,
        CASE 
            WHEN income.annual_income < 15000 THEN 'Lower'
            WHEN income.annual_income >= 16000 AND income.annual_income < 75000 THEN 'Middle'
            ELSE 'Upper'
        END AS income_level
        FROM person
        JOIN income ON person.ssn = income.ssn;
    """)
    #execute query 
    income_level = pd.read_sql(query, connection)

print(income_level)

                     name        ssn  annual_income income_level
0      Christoper Peteuil  747714076          31000       Middle
1     Kourtney Calderwood  477972044          24000       Middle
2               Muoi Cary  828638512          14800        Lower
3             Era Moselle  614621061          47400       Middle
4         Antione Godbolt  491650087          79300        Upper
...                   ...        ...            ...          ...
7509          Luba Benser  685095054          35100       Middle
7510      Roxana Mckimley  512136801          80100        Upper
7511      Cherie Zeimantz  362877324          70200       Middle
7512          Allen Cruse  348734531          78500        Upper
7513         Vance Hunten  896677562          11000        Lower

[7514 rows x 4 columns]


In [81]:
#create a place to store how many of each income levels exist 
count_income_levels = income_level['income_level'].value_counts()

#print a clean list of each income level 
for level, count in count_income_levels.items():
    print(f"{level}    {count}")

Middle    5467
Upper    1577
Lower    470


**CASE STATEMENT**
<br>
Write a second CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** What type of dirvers license does each person have in SQL City (assuming everyone followed the typical route of passin road test and written test at 16 or 17)? 
<br>
**What tables are we joining? (If any):** person and drivers license 

In [23]:
#make a connection to the database using the engine
with engine.connect() as connection:
    #define the query to join the people and drivers license tables and then classify each person's license as probationary (< 18), Restricted Basic (>= 18 and , > 21), or  non Restricted Basic (>21) 
    query = text("""
        SELECT
            person.name,
            person.license_id,
            drivers_license.age,
            drivers_license.id AS license_id,
        CASE 
            WHEN drivers_license.age < 18 THEN 'Probationary'
            WHEN drivers_license.age >= 18 AND drivers_license.age < 21 THEN 'Restricted Basic' 
            ELSE 'Non Restricted Basic'
        END AS license_type
        FROM person
        JOIN drivers_license ON person.license_id = drivers_license.id;
    """)
    #execute query 
    license_type = pd.read_sql(query, connection)

print(license_type)

                      name  license_id  age  license_id          license_type
0       Christoper Peteuil      993845   46      993845  Non Restricted Basic
1      Kourtney Calderwood      861794   54      861794  Non Restricted Basic
2                Muoi Cary      385336   24      385336  Non Restricted Basic
3              Era Moselle      431897   22      431897  Non Restricted Basic
4             Trena Hornby      550890   45      550890  Non Restricted Basic
...                    ...         ...  ...         ...                   ...
10001          Luba Benser      274427   26      274427  Non Restricted Basic
10002      Roxana Mckimley      975942   87      975942  Non Restricted Basic
10003      Cherie Zeimantz      287627   29      287627  Non Restricted Basic
10004          Allen Cruse      251350   66      251350  Non Restricted Basic
10005         Vance Hunten      830407   61      830407  Non Restricted Basic

[10006 rows x 5 columns]


In [25]:
#create a place to store how many of each license types exist 
count_license = license_type['license_type'].value_counts()

#print a clean list of each type 
for level, count in count_license.items():
    print(f"{level}    {count}")

Non Restricted Basic    9581
Restricted Basic    425


**CASE STATEMENT**
<br>
Write a third CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** Are males in SQL city tall, short, or average height?
<br>
**What tables are we joining? (If any):** Person and drivers license 

In [27]:
#make a connection to the database using the engine
with engine.connect() as connection:
    #define the query to classify male height as short( <60 in), ave (>= 60 in but <= 74 in), tall (<74)
    query = text("""
        SELECT 
            person.name,
            person.license_id,
            drivers_license.gender,
            drivers_license.id AS license_id,
            drivers_license.height, 
        CASE 
            WHEN drivers_license.gender = 'male' AND drivers_license.height < 60 THEN 'Short'
            WHEN drivers_license.gender = 'male' AND drivers_license.height BETWEEN 60 AND 74 THEN 'Average'
            ELSE 'Tall'
        END AS height_category
        FROM person
        JOIN drivers_license ON person.license_id = drivers_license.id;
    """)
    male_height_category = pd.read_sql(query, connection)

male_height_category


,name,license_id,gender,license_id,height,height_category
0,Christoper Peteuil,993845,male,993845,59,Short
1,Kourtney Calderwood,861794,female,861794,74,Tall
2,Muoi Cary,385336,female,385336,79,Tall
3,Era Moselle,431897,female,431897,75,Tall
4,Trena Hornby,550890,female,550890,60,Tall
...,...,...,...,...,...,...
10001,Luba Benser,274427,female,274427,55,Tall
10002,Roxana Mckimley,975942,female,975942,54,Tall
10003,Cherie Zeimantz,287627,female,287627,58,Tall
10004,Allen Cruse,251350,male,251350,61,Average


In [29]:
#create a place to store how many of each license types exist 
count_height = male_height_category['height_category'].value_counts()

#print a clean list of each type 
for level, count in count_height.items():
    print(f"{level}    {count}")

Tall    6210
Average    2267
Short    1529


**NULL**
<br>
As with all data, NULL values are fields with no data. Null data can be useful to know with the INSERT INTO statement below.

Find the NULL data within your database. Write your question you are answering with your data query. <br>
<br>
**Question:**  Is there anyone who did not give an interview?
<br>

In [31]:
with engine.connect() as connection:
    #define query to join the person table with the interview table to list all people that do not have an interview
    query = text("""
        SELECT 
            person.id, 
            person.name,
            interview.person_id, 
            interview.transcript
        FROM person
        LEFT JOIN interview ON person.id = interview.person_id
        WHERE interview.transcript IS NULL;
    """)
    no_interview = pd.read_sql(query, connection)

print(no_interview)

         id                name person_id transcript
0     10000  Christoper Peteuil      None       None
1     10010           Muoi Cary      None       None
2     10126      Denver Barness      None       None
3     10127     Yessenia Fossen      None       None
4     10145      Adolfo Milbury      None       None
...     ...                 ...       ...        ...
5015  99924      Palmer Servano      None       None
5016  99928       Jade Gardenas      None       None
5017  99941     Roxana Mckimley      None       None
5018  99982         Allen Cruse      None       None
5019  99990        Vance Hunten      None       None

[5020 rows x 4 columns]


There are **over 5,000 people** who did not give interviews.

**INSERT INTO**
<br>
You can insert new records into a table as needed using the INSERT INTO statement. If you choose to populate a table with certain records and not others, the rest of the fields will remain empty/NULL.
<br>
For INSERT INTO, we are not querying the database, instead we are ADDING to it. We do not need to use dbGetQuery(), but instead, dbExecute()!
<br><br>
**Objective:** Pick 10 people and add this line: "I do not wish to provide an interview" in the interview table. 
<br>
**What table(s) are we adding a record to?** interview 

In [112]:
#make a list of ids to add to the person id column 
person_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

with engine.connect() as connection:
    #insert the transcript for each person 
    for person_id in person_id:
        query = text("""
            INSERT INTO interview (person_id, transcript)
            VALUES (:person_id, :transcript)
        """)
        #execute adding the text for the transcript column
        connection.execute(query, {
            "person_id": person_id,
            "transcript": "I do not wish to provide an interview."})

print("Inserted 10 interview records with opt-out message.")

Inserted 10 interview records with opt-out message.


**MIN() and MAX()**
<br>
You can use these statements alone or in combination with the CASE statemts above.<br>
The IN operator in a WHERE clause lets you filter for multiple values at once. You can also exclude certain values by using the NOT IN operator.
<br>

**Question:** What is the tallest height for males and females?
<br>
**What table(s) are we joining? (If any)** None, just using the drivers license table

In [57]:
with engine.connect() as connection:
    #define the query to find the max height for both females and males using the drivers license table 
    query = text("""
        SELECT drivers_license.gender,
            MAX(CASE WHEN drivers_license.gender = 'male' THEN drivers_license.height ELSE NULL END) as max_male_height,
            MAX(CASE WHEN drivers_license.gender = 'female' THEN drivers_license.height ELSE NULL END) as max_female_height
        FROM drivers_license
        GROUP BY gender
    """)

    max_heights = pd.read_sql(query, connection)

print(max_heights)


   gender  max_male_height  max_female_height
0  female              NaN               82.0
1    male             84.0                NaN


The max height for males in SQL city is **84 inches or 7ft** and for females it is **82 in or 6'8" ft**. 


Combine CASE statement with Min() and Max() for a more detailed query of your data:
<br><br>
**Question:** What is the tallest and shortest height for both males and females in SQL City? Find the names of the people who are either the tallest or shortest. 
<br>
**What table(s) are we joining?** person and drivers license 

In [90]:
with engine.connect() as connection:
    #define the query to find the max and min heights for both females and males using the drivers license table 
    query = text("""
        SELECT drivers_license.gender,
            MAX(CASE WHEN drivers_license.gender = 'male' THEN drivers_license.height ELSE NULL END) as max_male_height,
            MAX(CASE WHEN drivers_license.gender = 'female' THEN drivers_license.height ELSE NULL END) as max_female_height,
            MIN(CASE WHEN drivers_license.gender = 'male' THEN drivers_license.height ELSE NULL END) as min_male_height,
            MIN(CASE WHEN drivers_license.gender = 'female' THEN drivers_license.height ELSE NULL END) as min_female_height
        FROM drivers_license
        GROUP BY gender
    """)

    heights = pd.read_sql(query, connection)

print(heights)


   gender  max_male_height  max_female_height  min_male_height  \
0  female              NaN               82.0              NaN   
1    male             84.0                NaN             50.0   

   min_female_height  
0               50.0  
1                NaN  


In [79]:
with engine.connect() as connection:
    #define the query to find the names of each person who is the tallest for both genders 
    #change the drivers_license.height IN (50)) to either the max or min height for the specific gender
    query = text("""
        SELECT 
            person.name,
            drivers_license.gender,
            drivers_license.height
        FROM person
        JOIN drivers_license ON person.license_id = drivers_license.id
        WHERE 
            (drivers_license.gender = 'male' AND drivers_license.height IN (50))
            OR
            (drivers_license.gender = 'female' AND drivers_license.height IN (50));
    """)

    names_for_theights = pd.read_sql(query, connection)

print(names_for_theights)


                  name  gender  height
0         Daren Cwikla    male      50
1         Jovita Crear  female      50
2         Hubert Yeddo    male      50
3       Sunni Scadlock  female      50
4      Guillermo Janda    male      50
..                 ...     ...     ...
291      Mirna Monette  female      50
292   Maria Swartzbeck    male      50
293   Anisa Woodbridge  female      50
294    Nguyet Marcinka  female      50
295  Edwardo Dicamillo    male      50

[296 rows x 3 columns]


There are **157** people that are the **tallest** females and males living in SQL City. 
<br>
There are **296** people that are the **shortest** males and females in SQL City. 

**MIN() and MAX()** <br>
AVG() will take the average of a numeric field.

**Question:** What is the average height of males and females? 
<br>

In [96]:
with engine.connect() as connection:
    #define the query to find the ave height for both females and males using the drivers license table 
    query = text("""
        SELECT drivers_license.gender,
            AVG(CASE WHEN drivers_license.gender = 'male' THEN drivers_license.height ELSE NULL END) as ave_male_height,
            AVG(CASE WHEN drivers_license.gender = 'female' THEN drivers_license.height ELSE NULL END) as ave_female_height
        FROM drivers_license
        GROUP BY gender
    """)

    ave_heights = pd.read_sql(query, connection)

print(ave_heights)

   gender  ave_male_height  ave_female_height
0  female              NaN          66.255332
1    male        65.853328                NaN


The average male height is **65in or 5'4"** and the average female height is **66.25in or 5'5"**. Surpringly in SQL City on average women are taller then men. 

Although this query gives us a quick answer, as a stand alone data frame, it is not that useful. We can add it to our longer query above. <br><br>
**Aliases (AS)**
<br>
You can abbreviate your code to make it more visually appealing...or more confusing? :) <br>
<br>
Examples:<br>
FROM table_name t<br>
FROM table_name AS t<br>
<br>
**Objective:** Use abbreviations or aliases for all tables for the same code you wrote above (If you have not done so already). Be sure to obtain the same result set.

In [102]:
with engine.connect() as connection:
    #define the query to find the max height for both females and males using the drivers license table 
    query = text("""
        SELECT dl.gender,
            AVG(CASE WHEN dl.gender = 'male' THEN dl.height ELSE NULL END) as ave_male_height,
            AVG(CASE WHEN dl.gender = 'female' THEN dl.height ELSE NULL END) as ave_female_height
        FROM drivers_license AS dl
        GROUP BY dl.gender
    """)

    ave_heights = pd.read_sql(query, connection)

print(ave_heights)

   gender  ave_male_height  ave_female_height
0  female              NaN          66.255332
1    male        65.853328                NaN


Now we are starting to create multiple new fields that we can save any time as a .csv if needed to access later. Save your result set as a .csv:

In [112]:
# Save result set as .csv file:
ave_heights.to_csv(r"C:\Users\parni\OneDrive\Desktop\DSSA\DGW\ave_heights.csv", index=False)

print(ave_heights)

   gender  ave_male_height  ave_female_height
0  female              NaN          66.255332
1    male        65.853328                NaN


**DELETE** ~Caution!~
<br>
You can delete all records from specific tables or set a criteria to delete certain values or NULL values without deleting the table itself. It is okay if you do not execute the code if you have completed all data cleaning steps earlier in the semester.<br>
<br>
If you created autoincrement IDs for any of your data, it is recommended to use TRUNCATE TABLE instead, used the same way. The ID's will automatically reset if needed.<br>
<br>
**Objective:** Delete all the data from the interview table for the SQL Symphony concert. 
<br>
**What table(s) are we deleting records from?** facebook_event_checkin

In [148]:
with engine.connect() as connection:
    #define the query to dleete all rows where the concert name was SQL Symphony concert 
    delete_query = text("""
        DELETE FROM facebook_event_checkin
        WHERE event_name = "SQL Symphony Concert";
    """)
    connection.execute(delete_query)
    connection.commit()
    print("All check-ins for the SQL Symphony Concert have been deleted.")

All check-ins for the SQL Symphony Concert have been deleted.


In [154]:
 with engine.connect() as connection:
    #define the query to check if the rows have been deleted 
    query = text("""
        SELECT *
        FROM facebook_event_checkin
        WHERE event_name = 'SQL Symphony Concert';
    """)
    deleted_rows = pd.read_sql(query, connection)

print(deleted_rows)
print(f"Sucess!")


Empty DataFrame
Columns: [person_id, event_id, event_name, date]
Index: []
Sucess!


We can also delete entire tables in MySQL workbench by manually right clicking on the table and DROP TABLE. <br>
MySQL Workbench will prompt you to review the SQL syntax before dropping the table.<br>
The syntax is simple:<br>
DROP TABLE table_name<br>

In [ ]:
#Close the database connection :)
connection.close()

**STOP**<br>
Before you submit, did you comment all your code?<br>
Did you answer all of the questions in the markdown cells?<br>
Did you rename the file and write your name at the top of the .pynb?<br>
Attach the .csv file you created with your Blackboard submission. It is preferred that you submit your Github link instead of the file itself.